# Prerequisites

# Introduction
 (motivation)


## What will be covered in this notebook?

## Core concepts
    - evaluating generated text
    - fairness and bias in AI/ML
    - Importance of being use-case specific over benchmark
## The Task
    - set up the immediate problem we are solving with this notebook

# Fairness and Bias Evaluation Workflow

(Diagram here?)

# Set up

In [ ]:
import os
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
MISTRAL_API_KEY = userdata.get('MISTRAL_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY

# Plan the evaluation approach

# Generate an evaluation dataset

# Run evaluation

# Gut-check your evaluation

# What's Next?

# Resources